## 数据预处理

In [73]:
from datasets import load_dataset
train_file = 'dataset\\atis\\atis.train.iob'
valid_file = 'dataset\\atis\\atis.test.iob'

atis_dataset = load_dataset("text", data_files={"train": train_file, "validate": valid_file})

Using custom data configuration default-52cb1d50be8a857b
Reusing dataset text (F:\cachefile\huggingface\datasets\text\default-52cb1d50be8a857b\0.0.0\21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad)


  0%|          | 0/2 [00:00<?, ?it/s]

In [74]:
atis_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4978
    })
    validate: Dataset({
        features: ['text'],
        num_rows: 893
    })
})

In [75]:
len(atis_dataset["train"])

4978

In [40]:
list3 = []
list1 = ["a", "b", "c"]
list2 = ["a", "d", "c"]
list = list3 + list1 + list2
myset = set(list)

In [41]:
myset

{'a', 'b', 'c', 'd'}

### 抽取类别标签

In [76]:
def extract_labels():
    train_data = atis_dataset["train"]
    test_data = atis_dataset["validate"]
    label_list = []
    for i in range(len(train_data)):
        list = train_data[i]["text"].split("\t")
        tags_name = list[1].split(" ")
        tags_name.pop(0)
        tags_name.pop()
        label_list = label_list + tags_name
    for i in range(len(test_data)):
        list = test_data[i]["text"].split("\t")
        tags_name = list[1].split(" ")
        tags_name.pop(0)
        tags_name.pop()
        label_list = label_list + tags_name
    labels = set(label_list)
    return labels

In [77]:
labels = extract_labels()

In [78]:
len(labels)

127

In [79]:
labels_list = [*labels, ]

In [80]:
len(labels_list)

127

### 编码类别标签

In [81]:
from sklearn.preprocessing import LabelEncoder
labels_set = extract_labels()
labels = [*labels_set, ]
label_encoder = LabelEncoder()
label_encoder.fit(labels)

LabelEncoder()

In [82]:
label_encoder.classes_

array(['B-aircraft_code', 'B-airline_code', 'B-airline_name',
       'B-airport_code', 'B-airport_name', 'B-arrive_date.date_relative',
       'B-arrive_date.day_name', 'B-arrive_date.day_number',
       'B-arrive_date.month_name', 'B-arrive_date.today_relative',
       'B-arrive_time.end_time', 'B-arrive_time.period_mod',
       'B-arrive_time.period_of_day', 'B-arrive_time.start_time',
       'B-arrive_time.time', 'B-arrive_time.time_relative',
       'B-booking_class', 'B-city_name', 'B-class_type', 'B-compartment',
       'B-connect', 'B-cost_relative', 'B-day_name', 'B-day_number',
       'B-days_code', 'B-depart_date.date_relative',
       'B-depart_date.day_name', 'B-depart_date.day_number',
       'B-depart_date.month_name', 'B-depart_date.today_relative',
       'B-depart_date.year', 'B-depart_time.end_time',
       'B-depart_time.period_mod', 'B-depart_time.period_of_day',
       'B-depart_time.start_time', 'B-depart_time.time',
       'B-depart_time.time_relative', 'B-econom

### 添加tags列

In [83]:
def add_tags_column(item):
    list = item["text"].split("\t")
    text_list = list[0].split(" ")
    text_list.pop(0)
    text_list.pop()
    tags_name_list = list[1].split(" ")
    tags_name_list.pop(0)
    tags_name_list.pop()
    item["text"] = text_list
    item['tags_name'] = tags_name_list
    item['tags'] = label_encoder.transform(tags_name_list).tolist()
    return item

In [84]:
atis_dataset["train"][0]

{'text': 'BOS i want to fly from boston at 838 am and arrive in denver at 1110 in the morning EOS\tO O O O O O B-fromloc.city_name O B-depart_time.time I-depart_time.time O O O B-toloc.city_name O B-arrive_time.time O O B-arrive_time.period_of_day O'}

In [85]:
item = add_tags_column(atis_dataset["train"][0])

In [86]:
item

{'text': ['i',
  'want',
  'to',
  'fly',
  'from',
  'boston',
  'at',
  '838',
  'am',
  'and',
  'arrive',
  'in',
  'denver',
  'at',
  '1110',
  'in',
  'the',
  'morning'],
 'tags_name': ['O',
  'O',
  'O',
  'O',
  'O',
  'B-fromloc.city_name',
  'O',
  'B-depart_time.time',
  'I-depart_time.time',
  'O',
  'O',
  'O',
  'B-toloc.city_name',
  'O',
  'B-arrive_time.time',
  'O',
  'O',
  'B-arrive_time.period_of_day'],
 'tags': [126,
  126,
  126,
  126,
  126,
  48,
  126,
  35,
  99,
  126,
  126,
  126,
  78,
  126,
  14,
  126,
  126,
  12]}

In [87]:
len(item["text"])

18

In [88]:
len(item["tags_name"])

18

In [89]:
len(item["tags"])

18

In [90]:
atis_dataset_taged = atis_dataset.map(add_tags_column)

  0%|          | 0/4978 [00:00<?, ?ex/s]

  0%|          | 0/893 [00:00<?, ?ex/s]

In [91]:
atis_dataset_taged

DatasetDict({
    train: Dataset({
        features: ['text', 'tags_name', 'tags'],
        num_rows: 4978
    })
    validate: Dataset({
        features: ['text', 'tags_name', 'tags'],
        num_rows: 893
    })
})

In [92]:
atis_dataset_taged["train"][0:3]

{'text': [['i',
   'want',
   'to',
   'fly',
   'from',
   'boston',
   'at',
   '838',
   'am',
   'and',
   'arrive',
   'in',
   'denver',
   'at',
   '1110',
   'in',
   'the',
   'morning'],
  ['what',
   'flights',
   'are',
   'available',
   'from',
   'pittsburgh',
   'to',
   'baltimore',
   'on',
   'thursday',
   'morning'],
  ['what',
   'is',
   'the',
   'arrival',
   'time',
   'in',
   'san',
   'francisco',
   'for',
   'the',
   '755',
   'am',
   'flight',
   'leaving',
   'washington']],
 'tags_name': [['O',
   'O',
   'O',
   'O',
   'O',
   'B-fromloc.city_name',
   'O',
   'B-depart_time.time',
   'I-depart_time.time',
   'O',
   'O',
   'O',
   'B-toloc.city_name',
   'O',
   'B-arrive_time.time',
   'O',
   'O',
   'B-arrive_time.period_of_day'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'B-fromloc.city_name',
   'O',
   'B-toloc.city_name',
   'O',
   'B-depart_date.day_name',
   'B-depart_time.period_of_day'],
  ['O',
   'O',
   'O',
   'B-flight_time',
  

## 保存数据集

In [93]:
atis_dataset_taged.save_to_disk("dataset\\atis\\atis")